In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
# Step 1: Load your training data
train_df = pd.read_csv('train.csv')  

# Define your features (X) and target (y) from the training data
X = train_df.drop(columns=['num_sold'])  # All columns except the target
y = train_df['num_sold']  # Target variable

In [3]:
y = y.fillna(y.mean())

In [4]:
y.isnull().sum()

np.int64(0)

In [5]:
# Step 2: Preprocessing (handling categorical variables, missing values)
preprocessor = ColumnTransformer(
    transformers=[  
        # Numerical features: Impute missing values (no scaling)
        ('num', Pipeline([ 
            ('imputer', SimpleImputer(strategy='mean'))  # Handle missing values
        ]), ['id']),  # Replace with actual numerical column names

        # Categorical features: OneHotEncoder
        ('cat', OneHotEncoder(handle_unknown='ignore'), [ 'date','country', 'store', 'product'])  # Categorical columns
    ])

In [6]:
# Step 3: Random Forest Model
model_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(
        n_estimators=100,  # Number of trees
        max_depth=None,    # No maximum depth for trees
        random_state=42,   # For reproducibility
        n_jobs=-1          # Use all available processors
    ))
])

In [ ]:
# Step 4: Train the model on the full dataset
model_rf.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['id']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['date', 'country', 'store',
                                                   'product'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

In [11]:
# Step 5: Evaluate the model on the training data (since no train-test split is performed here)
y_pred = model_rf.predict(X)

# Calculate R^2 score and MAPE for training data
r2_train = model_rf.score(X, y)
mape_train = mean_absolute_percentage_error(y, y_pred)

print(f"R2 Score: {r2_train:.4f}")
print(f"MAPE: {mape_train:.4f}")

R2 Score: 0.9961
MAPE: 0.1314


In [9]:
test_df = pd.read_csv('test.csv')
test_predictions = model_rf.predict(test_df)

In [10]:
test_df['num_sold'] = test_predictions
submission = test_df[['id', 'num_sold']]
submission.to_csv("Submission_file.csv", index=False)
print("Predictions saved to 'Submission_file.csv'.")

Predictions saved to 'Submission_file.csv'.
